In [1]:
import os
import pickle
import importlib
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn.functional as F

import sys
sys.path.append('..')

import grb

# Data preparation

## Load data

In [2]:
data_dir = "/data/qinkai/aminer_revised/"
with open(os.path.join(data_dir, "adj.pkl"), 'rb') as f:
    adj = pickle.load(f)
labels = np.load(os.path.join(data_dir, "labels_train.npy"))
features = np.load(os.path.join(data_dir, "features.npy"))

In [3]:
# padding unknown test labels by zeros
# labels = np.concatenate([labels, np.zeros(features.shape[0] - labels.shape[0])], axis=0)

# padding unknown test labels by true labels
labels_test = np.load(os.path.join(data_dir, "labels_test.npy"))
labels = np.concatenate([labels, labels_test], axis=0)

## Build customized dataset

In [4]:
from grb.dataset.dataset import CustomDataset
from grb.utils import fix_seed, get_num_params

In [5]:
n_node = features.shape[0]
n_val = 50000  # user-defined val size
n_test = 50000
train_mask = torch.zeros(n_node, dtype=bool)
train_mask[range(n_node - n_val - n_test)] = True
val_mask = torch.zeros(n_node, dtype=bool)
val_mask[range(n_node - n_val - n_test, n_node - n_test)] = True
test_mask = torch.zeros(n_node, dtype=bool)
test_mask[range(n_node - n_test, n_node)] = True

In [6]:
dataset = CustomDataset(adj=adj,
                        features=features,
                        labels=labels,
                        train_mask=train_mask,
                        val_mask=val_mask,
                        test_mask=test_mask,
                        name='Aminer')

Custom Dataset 'Aminer' loaded.
    Number of nodes: 659574.
    Number of edges: 5757154.
    Number of features: 100.
    Number of classes: 18.
    Number of train samples: 559574.
    Number of val samples: 50000.
    Number of test samples: 50000.
    Feature range [-0.6672, 0.6483]


In [7]:
adj = dataset.adj
adj_tensor = dataset.adj_tensor
features = dataset.features
labels = dataset.labels
num_features = dataset.num_features
num_classes = dataset.num_classes

## Fix random seed

In [9]:
# fix_seed(seed=0)

# Model training

## GCN

### Build model

In [8]:
from grb.model.gcn import GCN

model = GCN(in_features=num_features, 
            out_features=num_classes, 
            hidden_features=[256, 128, 64], 
            activation=F.relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 68178.
GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=100, out_features=256, bias=True)
    )
    (1): GCNConv(
      (linear): Linear(in_features=256, out_features=128, bias=True)
    )
    (2): GCNConv(
      (linear): Linear(in_features=128, out_features=64, bias=True)
    )
    (3): GCNConv(
      (linear): Linear(in_features=64, out_features=18, bias=True)
    )
  )
)


### Build model (with LayerNorm)

In [8]:
from grb.model.gcn import GCN

model = GCN(in_features=num_features, 
            out_features=num_classes, 
            hidden_features=[256, 128, 64], 
            layer_norm=True,
            activation=F.relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 69146.
GCN(
  (layers): ModuleList(
    (0): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (1): GCNConv(
      (linear): Linear(in_features=100, out_features=256, bias=True)
    )
    (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (3): GCNConv(
      (linear): Linear(in_features=256, out_features=128, bias=True)
    )
    (4): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (5): GCNConv(
      (linear): Linear(in_features=128, out_features=64, bias=True)
    )
    (6): GCNConv(
      (linear): Linear(in_features=64, out_features=18, bias=True)
    )
  )
)


### Prepare trainer

In [9]:
from grb.model.trainer import Trainer
from grb.utils.normalize import GCNAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=GCNAdjNorm,
                  lr_scheduler=False,
                  early_stop=False,
                  device='cuda:4')

### Model training

In [ ]:
trainer.train(model=model,
              n_epoch=12000,
              save_dir="./saved_models/gcn_aminer",
              eval_every=100,
              save_after=1000,
              dropout=0.5,
              verbose=True)

Epoch 00000 | Train Loss 2.8837 | Train Acc 0.0239 | Val Loss 2.8838 | Val Acc 0.0237
Epoch 00100 | Train Loss 1.3392 | Train Acc 0.5981 | Val Loss 1.3529 | Val Acc 0.5931
Epoch 00200 | Train Loss 1.2148 | Train Acc 0.6283 | Val Loss 1.2258 | Val Acc 0.6230
Epoch 00300 | Train Loss 1.1707 | Train Acc 0.6384 | Val Loss 1.1849 | Val Acc 0.6341
Epoch 00400 | Train Loss 1.1412 | Train Acc 0.6465 | Val Loss 1.1577 | Val Acc 0.6417
Epoch 00500 | Train Loss 1.1259 | Train Acc 0.6504 | Val Loss 1.1466 | Val Acc 0.6428
Epoch 00600 | Train Loss 1.1106 | Train Acc 0.6535 | Val Loss 1.1321 | Val Acc 0.6478
Epoch 00700 | Train Loss 1.1017 | Train Acc 0.6563 | Val Loss 1.1245 | Val Acc 0.6508
Epoch 00800 | Train Loss 1.0945 | Train Acc 0.6575 | Val Loss 1.1214 | Val Acc 0.6502
Epoch 00900 | Train Loss 1.0876 | Train Acc 0.6588 | Val Loss 1.1120 | Val Acc 0.6535
Epoch 01000 | Train Loss 1.0803 | Train Acc 0.6616 | Val Loss 1.1071 | Val Acc 0.6530
Best validation accuracy: 0.6554
Model saved in './sav

### Load trained model

In [10]:
model.load_state_dict(torch.load('./saved_models/gcn_ln_aminer/checkpoint_final.pt'))

<All keys matched successfully>

In [11]:
logits, test_acc = trainer.inference(model)
print("Test ACC: {:.4f}".format(test_acc))

Test ACC: 0.6835


## TAGCN

### Build model

In [10]:
from grb.utils import fix_seed, get_num_params

fix_seed(seed=0)  # fix random seed

In [14]:
from grb.model.tagcn import TAGCN

model = TAGCN(in_features=num_features, out_features=num_classes, hidden_features=[128, 128, 128], k=2, 
              activation=F.leaky_relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 144018.
TAGCN(
  (layers): ModuleList(
    (0): TAGConv(
      (linear): Linear(in_features=300, out_features=128, bias=True)
    )
    (1): TAGConv(
      (linear): Linear(in_features=384, out_features=128, bias=True)
    )
    (2): TAGConv(
      (linear): Linear(in_features=384, out_features=128, bias=True)
    )
    (3): TAGConv(
      (linear): Linear(in_features=384, out_features=18, bias=True)
    )
  )
)


### Prepare trainer

In [15]:
from grb.model.trainer import Trainer
from grb.utils.normalize import GCNAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.001)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=GCNAdjNorm,
                  lr_scheduler=True,
                  early_stop=True,
                  device='cuda:1')

### Model training

In [13]:
train_acc_list, val_acc_list = trainer.train(model=model,
                                             n_epoch=10000,
                                             save_dir="./saved_models/tagcn_aminer",
                                             eval_every=200,
                                             save_after=500,
                                             dropout=0.1,
                                             verbose=True)

Epoch 00000 | Train Loss 2.9431 | Train Acc 0.0580 | Val Loss 2.9444 | Val Acc 0.0577
Model saved in './saved_models/tagcn_aminer/checkpoint_epoch_0.pt'.
Epoch 00200 | Train Loss 1.2170 | Train Acc 0.6289 | Val Loss 1.2292 | Val Acc 0.6246
Epoch 00400 | Train Loss 1.1139 | Train Acc 0.6540 | Val Loss 1.1294 | Val Acc 0.6488
Model saved in './saved_models/tagcn_aminer/checkpoint_epoch_500.pt'.
Epoch 00600 | Train Loss 1.0611 | Train Acc 0.6663 | Val Loss 1.0770 | Val Acc 0.6625
Epoch 00800 | Train Loss 1.0271 | Train Acc 0.6745 | Val Loss 1.0489 | Val Acc 0.6679
Epoch 01000 | Train Loss 1.0054 | Train Acc 0.6805 | Val Loss 1.0296 | Val Acc 0.6744
Model saved in './saved_models/tagcn_aminer/checkpoint_epoch_1000.pt'.
Epoch 01200 | Train Loss 0.9888 | Train Acc 0.6852 | Val Loss 1.0196 | Val Acc 0.6762
Epoch  1355: reducing learning rate of group 0 to 7.5000e-04.
Epoch 01400 | Train Loss 0.9761 | Train Acc 0.6887 | Val Loss 1.0087 | Val Acc 0.6798
Model saved in './saved_models/tagcn_amin

### Load trained model

In [16]:
model.load_state_dict(torch.load('./saved_models/tagcn_aminer/checkpoint.pt'))

<All keys matched successfully>

In [17]:
logits, test_acc = trainer.inference(model)
print("Test ACC: {:.4f}".format(test_acc))

Test ACC: 0.7103


## GIN

### Build model

In [19]:
from grb.utils import fix_seed, get_num_params

fix_seed(seed=0)  # fix random seed

In [20]:
from grb.model.gin import GIN

model = GIN(in_features=num_features, out_features=num_classes, hidden_features=[128, 128, 128], activation=F.relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 114325.
GIN(
  (layers): ModuleList(
    (0): GINConv(
      (linear1): Linear(in_features=100, out_features=128, bias=True)
      (linear2): Linear(in_features=128, out_features=128, bias=True)
    )
    (1): GINConv(
      (linear1): Linear(in_features=128, out_features=128, bias=True)
      (linear2): Linear(in_features=128, out_features=128, bias=True)
    )
    (2): GINConv(
      (linear1): Linear(in_features=128, out_features=128, bias=True)
      (linear2): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (linear1): Linear(in_features=128, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=18, bias=True)
)


### Prepare trainer

In [21]:
from grb.model.trainer import Trainer
from grb.utils.normalize import GCNAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=GCNAdjNorm,
                  lr_scheduler=True,
                  early_stop=True,
                  device='cuda:0')

In [30]:
train_acc_list, val_acc_list = trainer.train(model=model,
                                             n_epoch=2000,
                                             save_dir="./saved_models/gin_aminer",
                                             eval_every=10,
                                             save_after=100,
                                             dropout=0.1,
                                             verbose=True)

Epoch 00000 | Train Loss 2.8910 | Train Acc 0.0148 | Val Loss 2.8911 | Val Acc 0.0158
Model saved in './saved_models/gin_aminer/checkpoint_epoch_0.pt'.
Epoch 00010 | Train Loss 2.6846 | Train Acc 0.1462 | Val Loss 2.6867 | Val Acc 0.1432
Epoch 00020 | Train Loss 2.5855 | Train Acc 0.1468 | Val Loss 2.5875 | Val Acc 0.1436
Epoch 00030 | Train Loss 2.5777 | Train Acc 0.1471 | Val Loss 2.5783 | Val Acc 0.1447
Epoch 00040 | Train Loss 2.5521 | Train Acc 0.1646 | Val Loss 2.5547 | Val Acc 0.1617
Epoch 00050 | Train Loss 2.5414 | Train Acc 0.2017 | Val Loss 2.5426 | Val Acc 0.1990
Epoch 00060 | Train Loss 2.4400 | Train Acc 0.2216 | Val Loss 2.4418 | Val Acc 0.2204
Epoch 00070 | Train Loss 2.3860 | Train Acc 0.2524 | Val Loss 2.3885 | Val Acc 0.2509
Epoch 00080 | Train Loss 2.2819 | Train Acc 0.2860 | Val Loss 2.2880 | Val Acc 0.2851
Epoch 00090 | Train Loss 2.2652 | Train Acc 0.2960 | Val Loss 2.2673 | Val Acc 0.2956
Epoch 00100 | Train Loss 2.1392 | Train Acc 0.3406 | Val Loss 2.1439 | Val

Epoch 00870 | Train Loss 1.0568 | Train Acc 0.6660 | Val Loss 1.0919 | Val Acc 0.6567
Epoch 00880 | Train Loss 1.0617 | Train Acc 0.6638 | Val Loss 1.0959 | Val Acc 0.6558
Epoch 00890 | Train Loss 1.0547 | Train Acc 0.6658 | Val Loss 1.0886 | Val Acc 0.6571
Epoch 00900 | Train Loss 1.0555 | Train Acc 0.6658 | Val Loss 1.0917 | Val Acc 0.6565
Model saved in './saved_models/gin_aminer/checkpoint_epoch_900.pt'.
Epoch   908: reducing learning rate of group 0 to 4.2188e-03.
Epoch 00910 | Train Loss 1.0503 | Train Acc 0.6679 | Val Loss 1.0860 | Val Acc 0.6578
Epoch 00920 | Train Loss 1.0484 | Train Acc 0.6680 | Val Loss 1.0839 | Val Acc 0.6581
Epoch 00930 | Train Loss 1.0462 | Train Acc 0.6685 | Val Loss 1.0850 | Val Acc 0.6592
Epoch 00940 | Train Loss 1.0453 | Train Acc 0.6692 | Val Loss 1.0831 | Val Acc 0.6586
Epoch 00950 | Train Loss 1.0439 | Train Acc 0.6690 | Val Loss 1.0819 | Val Acc 0.6594
Epoch 00960 | Train Loss 1.0424 | Train Acc 0.6695 | Val Loss 1.0795 | Val Acc 0.6602
Epoch 0097

In [24]:
model.load_state_dict(torch.load('./saved_models/gin_aminer/checkpoint_final.pt'))

<All keys matched successfully>

In [25]:
logits, test_acc = trainer.inference(model)
print("Test ACC: {:.4f}".format(test_acc))

RuntimeError: CUDA out of memory. Tried to allocate 324.00 MiB (GPU 0; 23.70 GiB total capacity; 5.87 GiB already allocated; 34.56 MiB free; 5.92 GiB reserved in total by PyTorch)

## SGCN

### Build model

In [30]:
from grb.model.sgcn import SGCN

model = SGCN(in_features=num_features, out_features=num_classes, hidden_features=[128, 128, 128], activation=F.relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 48474.
SGCN(
  (batchnorm): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (in_conv): Linear(in_features=100, out_features=128, bias=True)
  (out_conv): Linear(in_features=128, out_features=18, bias=True)
  (layers): ModuleList(
    (0): SGConv(
      (linear): Linear(in_features=128, out_features=128, bias=True)
    )
    (1): SGConv(
      (linear): Linear(in_features=128, out_features=128, bias=True)
    )
  )
)


### Prepare trainer

In [33]:
adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=GCNAdjNorm,
                  lr_scheduler=True,
                  early_stop=True,
                  device='cuda:2')

In [34]:
trainer.train(model=model,
              n_epoch=2000,
              save_dir="./saved_models/sgcn_aminer",
              eval_every=10,
              save_after=100,
              dropout=0.5,
              verbose=True)

Epoch 00000 | Train Loss 2.7566 | Train Acc 0.1463 | Val Loss 2.7575 | Val Acc 0.1433
Model saved in './saved_models/sgcn_aminer/checkpoint_epoch_0.pt'.
Epoch 00010 | Train Loss 2.5920 | Train Acc 0.1553 | Val Loss 2.5936 | Val Acc 0.1525
Epoch 00020 | Train Loss 2.4091 | Train Acc 0.2529 | Val Loss 2.4101 | Val Acc 0.2517
Epoch 00030 | Train Loss 2.1119 | Train Acc 0.3375 | Val Loss 2.1189 | Val Acc 0.3371
Epoch 00040 | Train Loss 1.9016 | Train Acc 0.4171 | Val Loss 1.9082 | Val Acc 0.4155
Epoch 00050 | Train Loss 1.7275 | Train Acc 0.4784 | Val Loss 1.7401 | Val Acc 0.4732
Epoch 00060 | Train Loss 1.6219 | Train Acc 0.5176 | Val Loss 1.6326 | Val Acc 0.5143
Epoch 00070 | Train Loss 1.5498 | Train Acc 0.5431 | Val Loss 1.5614 | Val Acc 0.5399
Epoch 00080 | Train Loss 1.4836 | Train Acc 0.5632 | Val Loss 1.4990 | Val Acc 0.5608
Epoch 00090 | Train Loss 1.4391 | Train Acc 0.5761 | Val Loss 1.4507 | Val Acc 0.5733
Epoch 00100 | Train Loss 1.4104 | Train Acc 0.5856 | Val Loss 1.4205 | Va

Epoch 00880 | Train Loss 1.1438 | Train Acc 0.6525 | Val Loss 1.1590 | Val Acc 0.6504
Epoch 00890 | Train Loss 1.1444 | Train Acc 0.6522 | Val Loss 1.1627 | Val Acc 0.6468
Epoch 00900 | Train Loss 1.1433 | Train Acc 0.6529 | Val Loss 1.1601 | Val Acc 0.6463
Model saved in './saved_models/sgcn_aminer/checkpoint_epoch_900.pt'.
Epoch 00910 | Train Loss 1.1422 | Train Acc 0.6533 | Val Loss 1.1567 | Val Acc 0.6490
Epoch 00920 | Train Loss 1.1424 | Train Acc 0.6532 | Val Loss 1.1643 | Val Acc 0.6484
Epoch 00930 | Train Loss 1.1409 | Train Acc 0.6529 | Val Loss 1.1595 | Val Acc 0.6489
Epoch 00940 | Train Loss 1.1425 | Train Acc 0.6533 | Val Loss 1.1598 | Val Acc 0.6475
Epoch 00950 | Train Loss 1.1417 | Train Acc 0.6524 | Val Loss 1.1586 | Val Acc 0.6488
Epoch 00960 | Train Loss 1.1417 | Train Acc 0.6534 | Val Loss 1.1595 | Val Acc 0.6489
Epoch 00970 | Train Loss 1.1405 | Train Acc 0.6533 | Val Loss 1.1567 | Val Acc 0.6504
Epoch 00980 | Train Loss 1.1401 | Train Acc 0.6539 | Val Loss 1.1543 | 

Epoch 01730 | Train Loss 1.1128 | Train Acc 0.6596 | Val Loss 1.1296 | Val Acc 0.6546
Epoch 01740 | Train Loss 1.1123 | Train Acc 0.6604 | Val Loss 1.1272 | Val Acc 0.6564
Epoch 01750 | Train Loss 1.1112 | Train Acc 0.6604 | Val Loss 1.1274 | Val Acc 0.6552
Epoch 01760 | Train Loss 1.1116 | Train Acc 0.6606 | Val Loss 1.1311 | Val Acc 0.6559
Epoch 01770 | Train Loss 1.1113 | Train Acc 0.6598 | Val Loss 1.1326 | Val Acc 0.6548
Epoch  1779: reducing learning rate of group 0 to 1.3348e-03.
Epoch 01780 | Train Loss 1.1120 | Train Acc 0.6602 | Val Loss 1.1277 | Val Acc 0.6556
Epoch 01790 | Train Loss 1.1111 | Train Acc 0.6608 | Val Loss 1.1281 | Val Acc 0.6566
Epoch 01800 | Train Loss 1.1120 | Train Acc 0.6601 | Val Loss 1.1366 | Val Acc 0.6549
Model saved in './saved_models/sgcn_aminer/checkpoint_epoch_1800.pt'.
Epoch 01810 | Train Loss 1.1125 | Train Acc 0.6602 | Val Loss 1.1287 | Val Acc 0.6545
Epoch 01820 | Train Loss 1.1121 | Train Acc 0.6607 | Val Loss 1.1315 | Val Acc 0.6560
Epoch 01

## GraphSAGE

### Build model

In [8]:
from grb.model.graphsage import GraphSAGE

model = GraphSAGE(in_features=num_features, out_features=num_classes, 
                  hidden_features=[128, 128, 128], activation=F.relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 156184.
GraphSAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (pool_fc): Linear(in_features=100, out_features=100, bias=True)
      (fc1): Linear(in_features=100, out_features=128, bias=True)
      (fc2): Linear(in_features=100, out_features=128, bias=True)
    )
    (1): SAGEConv(
      (pool_fc): Linear(in_features=128, out_features=128, bias=True)
      (fc1): Linear(in_features=128, out_features=128, bias=True)
      (fc2): Linear(in_features=128, out_features=128, bias=True)
    )
    (2): SAGEConv(
      (pool_fc): Linear(in_features=128, out_features=128, bias=True)
      (fc1): Linear(in_features=128, out_features=128, bias=True)
      (fc2): Linear(in_features=128, out_features=128, bias=True)
    )
    (3): SAGEConv(
      (pool_fc): Linear(in_features=128, out_features=128, bias=True)
      (fc1): Linear(in_features=128, out_features=18, bias=True)
      (fc2): Linear(in_features=128, out_features=18, bias=True)
    )
  )
)


### Prepare trainer

In [10]:
from grb.model.trainer import Trainer
from grb.utils.normalize import SAGEAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=SAGEAdjNorm,
                  lr_scheduler=True,
                  early_stop=True,
                  device='cuda:2')

In [43]:
trainer.train(model=model,
              n_epoch=2000,
              save_dir="./saved_models/graphsage_aminer",
              eval_every=10,
              save_after=100,
              dropout=0.5,
              verbose=True)

Epoch 00000 | Train Loss 2.8781 | Train Acc 0.0612 | Val Loss 2.8774 | Val Acc 0.0624
Model saved in './saved_models/graphsage_aminer/checkpoint_epoch_0.pt'.
Epoch 00010 | Train Loss 2.5855 | Train Acc 0.1463 | Val Loss 2.5870 | Val Acc 0.1433
Epoch 00020 | Train Loss 2.5783 | Train Acc 0.1463 | Val Loss 2.5792 | Val Acc 0.1433
Epoch 00030 | Train Loss 2.5764 | Train Acc 0.1463 | Val Loss 2.5775 | Val Acc 0.1433
Epoch 00040 | Train Loss 2.5755 | Train Acc 0.1463 | Val Loss 2.5769 | Val Acc 0.1433
Epoch 00050 | Train Loss 2.5750 | Train Acc 0.1463 | Val Loss 2.5765 | Val Acc 0.1433
Epoch 00060 | Train Loss 2.5743 | Train Acc 0.1461 | Val Loss 2.5755 | Val Acc 0.1432
Epoch 00070 | Train Loss 2.5732 | Train Acc 0.1457 | Val Loss 2.5747 | Val Acc 0.1434
Epoch 00080 | Train Loss 2.5725 | Train Acc 0.1461 | Val Loss 2.5737 | Val Acc 0.1422
Epoch 00090 | Train Loss 2.5711 | Train Acc 0.1470 | Val Loss 2.5727 | Val Acc 0.1443
Epoch 00100 | Train Loss 2.5703 | Train Acc 0.1476 | Val Loss 2.5720

Epoch 00880 | Train Loss 1.1855 | Train Acc 0.6379 | Val Loss 1.2048 | Val Acc 0.6323
Epoch 00890 | Train Loss 1.1925 | Train Acc 0.6368 | Val Loss 1.2092 | Val Acc 0.6326
Epoch 00900 | Train Loss 1.1793 | Train Acc 0.6389 | Val Loss 1.1944 | Val Acc 0.6344
Model saved in './saved_models/graphsage_aminer/checkpoint_epoch_900.pt'.
Epoch 00910 | Train Loss 1.1676 | Train Acc 0.6431 | Val Loss 1.1843 | Val Acc 0.6380
Epoch 00920 | Train Loss 1.1664 | Train Acc 0.6442 | Val Loss 1.1877 | Val Acc 0.6377
Epoch 00930 | Train Loss 1.1629 | Train Acc 0.6446 | Val Loss 1.1773 | Val Acc 0.6394
Epoch 00940 | Train Loss 1.1649 | Train Acc 0.6436 | Val Loss 1.1780 | Val Acc 0.6389
Epoch 00950 | Train Loss 1.1644 | Train Acc 0.6437 | Val Loss 1.1808 | Val Acc 0.6383
Epoch 00960 | Train Loss 1.1571 | Train Acc 0.6462 | Val Loss 1.1750 | Val Acc 0.6402
Epoch 00970 | Train Loss 1.1560 | Train Acc 0.6467 | Val Loss 1.1735 | Val Acc 0.6417
Epoch 00980 | Train Loss 1.1561 | Train Acc 0.6464 | Val Loss 1.17

Epoch 01740 | Train Loss 1.1027 | Train Acc 0.6599 | Val Loss 1.1239 | Val Acc 0.6546
Epoch  1744: reducing learning rate of group 0 to 4.2188e-03.
Epoch 01750 | Train Loss 1.1032 | Train Acc 0.6607 | Val Loss 1.1225 | Val Acc 0.6537
Epoch 01760 | Train Loss 1.1028 | Train Acc 0.6599 | Val Loss 1.1215 | Val Acc 0.6539
Epoch 01770 | Train Loss 1.1004 | Train Acc 0.6610 | Val Loss 1.1219 | Val Acc 0.6548
Epoch 01780 | Train Loss 1.1013 | Train Acc 0.6608 | Val Loss 1.1216 | Val Acc 0.6546
Epoch 01790 | Train Loss 1.1013 | Train Acc 0.6611 | Val Loss 1.1255 | Val Acc 0.6541
Epoch 01800 | Train Loss 1.0990 | Train Acc 0.6615 | Val Loss 1.1205 | Val Acc 0.6540
Model saved in './saved_models/graphsage_aminer/checkpoint_epoch_1800.pt'.
Epoch 01810 | Train Loss 1.1019 | Train Acc 0.6610 | Val Loss 1.1253 | Val Acc 0.6547
Epoch 01820 | Train Loss 1.0993 | Train Acc 0.6618 | Val Loss 1.1183 | Val Acc 0.6560
Epoch 01830 | Train Loss 1.0997 | Train Acc 0.6615 | Val Loss 1.1221 | Val Acc 0.6546
Epo

TypeError: cannot unpack non-iterable NoneType object

In [13]:
model.load_state_dict(torch.load('./saved_models/graphsage_aminer/checkpoint.pt'))

<All keys matched successfully>

In [14]:
logits, test_acc = trainer.inference(model)
print("Test ACC: {:.4f}".format(test_acc))

Test ACC: 0.6677


## RobustGCN

### Build model

In [15]:
from grb.model.robustgcn import RobustGCN

model = RobustGCN(in_features=num_features, out_features=num_classes, 
                  hidden_features=[128, 128, 128])
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 96548.
RobustGCN(
  (layers): ModuleList(
    (0): RobustGCNConv(
      (mean_conv): Linear(in_features=100, out_features=128, bias=True)
      (var_conv): Linear(in_features=100, out_features=128, bias=True)
    )
    (1): RobustGCNConv(
      (mean_conv): Linear(in_features=128, out_features=128, bias=True)
      (var_conv): Linear(in_features=128, out_features=128, bias=True)
    )
    (2): RobustGCNConv(
      (mean_conv): Linear(in_features=128, out_features=128, bias=True)
      (var_conv): Linear(in_features=128, out_features=128, bias=True)
    )
    (3): RobustGCNConv(
      (mean_conv): Linear(in_features=128, out_features=18, bias=True)
      (var_conv): Linear(in_features=128, out_features=18, bias=True)
    )
  )
)


In [16]:
from grb.model.trainer import Trainer
from grb.utils.normalize import RobustGCNAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=RobustGCNAdjNorm,
                  lr_scheduler=True,
                  early_stop=True,
                  device='cuda:3')

In [12]:
trainer.train(model=model,
              n_epoch=5000,
              save_dir="./saved_models/robustgcn_aminer",
              eval_every=20,
              save_after=200,
              dropout=0.5,
              verbose=True)

Epoch 00000 | Train Loss 2.9001 | Train Acc 0.0351 | Val Loss 2.9000 | Val Acc 0.0342
Model saved in './saved_models/robustgcn_aminer/checkpoint_epoch_0.pt'.
Epoch 00020 | Train Loss 2.2813 | Train Acc 0.3303 | Val Loss 2.2832 | Val Acc 0.3264
Epoch 00040 | Train Loss 1.8688 | Train Acc 0.4453 | Val Loss 1.8695 | Val Acc 0.4441
Epoch 00060 | Train Loss 1.5769 | Train Acc 0.5349 | Val Loss 1.5804 | Val Acc 0.5328
Epoch 00080 | Train Loss 1.4389 | Train Acc 0.5694 | Val Loss 1.4461 | Val Acc 0.5662
Epoch 00100 | Train Loss 1.3806 | Train Acc 0.5857 | Val Loss 1.3899 | Val Acc 0.5830
Epoch 00120 | Train Loss 1.3256 | Train Acc 0.5977 | Val Loss 1.3336 | Val Acc 0.5931
Epoch 00140 | Train Loss 1.3007 | Train Acc 0.6035 | Val Loss 1.3094 | Val Acc 0.6004
Epoch 00160 | Train Loss 1.2783 | Train Acc 0.6078 | Val Loss 1.2856 | Val Acc 0.6047
Epoch 00180 | Train Loss 1.2639 | Train Acc 0.6119 | Val Loss 1.2747 | Val Acc 0.6079
Epoch 00200 | Train Loss 1.2534 | Train Acc 0.6143 | Val Loss 1.2632

([tensor(0.0351, device='cuda:3'),
  tensor(0.3303, device='cuda:3'),
  tensor(0.4453, device='cuda:3'),
  tensor(0.5349, device='cuda:3'),
  tensor(0.5694, device='cuda:3'),
  tensor(0.5857, device='cuda:3'),
  tensor(0.5977, device='cuda:3'),
  tensor(0.6035, device='cuda:3'),
  tensor(0.6078, device='cuda:3'),
  tensor(0.6119, device='cuda:3'),
  tensor(0.6143, device='cuda:3'),
  tensor(0.6172, device='cuda:3'),
  tensor(0.6202, device='cuda:3'),
  tensor(0.6218, device='cuda:3'),
  tensor(0.6245, device='cuda:3'),
  tensor(0.6247, device='cuda:3'),
  tensor(0.6268, device='cuda:3'),
  tensor(0.6280, device='cuda:3'),
  tensor(0.6295, device='cuda:3'),
  tensor(0.6296, device='cuda:3'),
  tensor(0.6314, device='cuda:3'),
  tensor(0.6320, device='cuda:3'),
  tensor(0.6338, device='cuda:3'),
  tensor(0.6348, device='cuda:3'),
  tensor(0.6342, device='cuda:3'),
  tensor(0.6362, device='cuda:3'),
  tensor(0.6371, device='cuda:3'),
  tensor(0.6360, device='cuda:3'),
  tensor(0.6380, dev

In [19]:
model.load_state_dict(torch.load('./saved_models/robustgcn_aminer/checkpoint.pt'))

FileNotFoundError: [Errno 2] No such file or directory: './saved_models/robustgcn_aminer/checkpoint_final.pt'

In [20]:
logits, test_acc = trainer.inference(model)
print("Test ACC: {:.4f}".format(test_acc))

Test ACC: 0.6801


In [24]:
np.where(adj.data > 0 & adj.data != 1)[0]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [33]:
np.where((adj.data > 0) * (adj.data == 1))[0]

array([      0,       1,       2, ..., 5757151, 5757152, 5757153])

In [34]:
adj.data[np.where((adj.data > 0) * (adj.data == 1))[0]] = 1

## APPNP

### Build model

In [11]:
from grb.model.appnp import APPNP

model = APPNP(in_features=num_features, out_features=num_classes, hidden_features=128, alpha=0.01, k=10)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 15250.
APPNP(
  (linear1): Linear(in_features=100, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=18, bias=True)
  (dropout): SparseEdgeDrop()
)


In [12]:
from grb.model.trainer import Trainer
from grb.utils.normalize import GCNAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=GCNAdjNorm,
                  lr_scheduler=True,
                  early_stop=None,
                  device='cuda:3')

In [13]:
trainer.train(model=model,
              n_epoch=5000,
              save_dir="./saved_models/appnp_aminer",
              eval_every=20,
              save_after=200,
              dropout=0.5,
              verbose=True)

Epoch 00000 | Train Loss 2.8816 | Train Acc 0.0964 | Val Loss 2.8816 | Val Acc 0.0985
Model saved in './saved_models/appnp_aminer/checkpoint_epoch_0.pt'.
Epoch 00020 | Train Loss 2.5242 | Train Acc 0.2423 | Val Loss 2.5274 | Val Acc 0.2377
Epoch 00040 | Train Loss 2.2119 | Train Acc 0.3471 | Val Loss 2.2148 | Val Acc 0.3449
Epoch 00060 | Train Loss 1.8402 | Train Acc 0.4924 | Val Loss 1.8431 | Val Acc 0.4898
Epoch 00080 | Train Loss 1.6149 | Train Acc 0.5458 | Val Loss 1.6192 | Val Acc 0.5431
Epoch 00100 | Train Loss 1.4967 | Train Acc 0.5763 | Val Loss 1.5019 | Val Acc 0.5728
Epoch 00120 | Train Loss 1.4406 | Train Acc 0.5899 | Val Loss 1.4465 | Val Acc 0.5869
Epoch 00140 | Train Loss 1.3977 | Train Acc 0.5984 | Val Loss 1.4017 | Val Acc 0.5946
Epoch 00160 | Train Loss 1.3709 | Train Acc 0.6065 | Val Loss 1.3769 | Val Acc 0.6025
Epoch 00180 | Train Loss 1.3546 | Train Acc 0.6087 | Val Loss 1.3601 | Val Acc 0.6060
Epoch 00200 | Train Loss 1.3394 | Train Acc 0.6124 | Val Loss 1.3421 | V

Epoch 01680 | Train Loss 1.2178 | Train Acc 0.6377 | Val Loss 1.2295 | Val Acc 0.6344
Epoch 01700 | Train Loss 1.2199 | Train Acc 0.6368 | Val Loss 1.2310 | Val Acc 0.6321
Epoch 01720 | Train Loss 1.2173 | Train Acc 0.6381 | Val Loss 1.2241 | Val Acc 0.6327
Epoch 01740 | Train Loss 1.2177 | Train Acc 0.6374 | Val Loss 1.2262 | Val Acc 0.6340
Epoch  1750: reducing learning rate of group 0 to 1.3348e-03.
Epoch 01760 | Train Loss 1.2151 | Train Acc 0.6379 | Val Loss 1.2268 | Val Acc 0.6333
Epoch 01780 | Train Loss 1.2179 | Train Acc 0.6382 | Val Loss 1.2284 | Val Acc 0.6350
Epoch  1801: reducing learning rate of group 0 to 1.0011e-03.
Epoch 01800 | Train Loss 1.2191 | Train Acc 0.6371 | Val Loss 1.2285 | Val Acc 0.6335
Model saved in './saved_models/appnp_aminer/checkpoint_epoch_1800.pt'.
Epoch 01820 | Train Loss 1.2155 | Train Acc 0.6371 | Val Loss 1.2236 | Val Acc 0.6338
Epoch 01840 | Train Loss 1.2164 | Train Acc 0.6363 | Val Loss 1.2259 | Val Acc 0.6322
Epoch  1852: reducing learning 

Epoch 03100 | Train Loss 1.2170 | Train Acc 0.6377 | Val Loss 1.2238 | Val Acc 0.6334
Epoch 03120 | Train Loss 1.2168 | Train Acc 0.6380 | Val Loss 1.2257 | Val Acc 0.6353
Epoch  3133: reducing learning rate of group 0 to 7.5339e-07.
Epoch 03140 | Train Loss 1.2183 | Train Acc 0.6369 | Val Loss 1.2291 | Val Acc 0.6340
Epoch 03160 | Train Loss 1.2156 | Train Acc 0.6371 | Val Loss 1.2238 | Val Acc 0.6334
Epoch 03180 | Train Loss 1.2173 | Train Acc 0.6371 | Val Loss 1.2266 | Val Acc 0.6339
Epoch  3184: reducing learning rate of group 0 to 5.6504e-07.
Epoch 03200 | Train Loss 1.2167 | Train Acc 0.6373 | Val Loss 1.2243 | Val Acc 0.6333
Model saved in './saved_models/appnp_aminer/checkpoint_epoch_3200.pt'.
Epoch 03220 | Train Loss 1.2168 | Train Acc 0.6382 | Val Loss 1.2235 | Val Acc 0.6336
Epoch  3235: reducing learning rate of group 0 to 4.2378e-07.
Epoch 03240 | Train Loss 1.2177 | Train Acc 0.6366 | Val Loss 1.2241 | Val Acc 0.6336
Epoch 03260 | Train Loss 1.2166 | Train Acc 0.6373 | Va

KeyboardInterrupt: 

In [16]:
model.load_state_dict(torch.load('./saved_models/appnp_aminer/checkpoint_epoch_3800.pt'))

<All keys matched successfully>

In [17]:
logits, test_acc = trainer.inference(model)
print("Test ACC: {:.4f}".format(test_acc))

Test ACC: 0.6444


## GAT

In [ ]:
from grb.model.gat import GAT

model = GAT(in_features=num_features, out_features=num_classes, hidden_features=128, alpha=0.01, k=10)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)